In [12]:
from http.server import SimpleHTTPRequestHandler, HTTPServer, BaseHTTPRequestHandler
import sys
import requests
from urllib.parse import urljoin, unquote
import urllib.parse
import numpy as np
import keras 
from keras.models import  load_model

In [13]:
!pip install --upgrade keras

In [14]:
import urllib.parse

# def url_decode(encoded_URL):
#     """
#     Decode the URL encoded string.
    
#     Parameters:
#         encoded_URL (str): The URL encoded string.
        
#     Returns:
#         str: The decoded URL string.
#     """
#     if isinstance(encoded_URL, str):  # Kiểm tra nếu tham số là một chuỗi
#         if encoded_URL != "/":  # Kiểm tra nếu giá trị không phải là "/"
#             decoded_url = urllib.parse.unquote(encoded_URL)
#             return decoded_url
#         else:
#             return encoded_URL  # Trả về giá trị gốc nếu là "/"
#     else:
#         # Trả về None hoặc thông báo lỗi tùy vào yêu cầu cụ thể của bạn
#         return None  # Hoặc raise TypeError("Invalid input type: expected str, got {type(encoded_URL)}")


# def load_data(urls, max_length=1000):
#     for url in urls:
#         if url != '/':  # Kiểm tra nếu giá trị không phải là '/'
#             try:
#                 url_lower = str(url).lower()
#             except AttributeError:  # Bắt ngoại lệ nếu giá trị không thể chuyển đổi thành chuỗi
#                 print(f"Skipping lowercase conversion for value: {url}")
#                 url_lower = url
#         else:
#             url_lower = url  # Giữ nguyên giá trị nếu là '/'
#     url_list = []
#     for url in urls:
#         # url decode
#         decoded_url = url_decode(url)
#         # unicode encode
#         encoded_url = [ord(x) for x in str(decoded_url).strip()]
#         encoded_url = encoded_url[:max_length]
#         url_len = len(encoded_url)
#         if url_len < max_length:
#             # zero padding
#             encoded_url += ([0] * (max_length - url_len))
#         url_list.append((encoded_url))
#     # convert to numpy array
#     url_list = np.array(url_list)
#     return url_list


def load_data(url, max_length=1000):
    # Chuyển đổi URL thành chữ thường
    url = url.lower()
    
    # Giải mã URL
    #decoded_url = url_decode(url)
    decoded_url = unquote(url)
    # Mã hóa Unicode
    encoded_url = [ord(x) for x in str(decoded_url).strip()]
    
    # Cắt ngắn URL nếu nó quá dài
    encoded_url = encoded_url[:max_length]
    
    # Thêm đệm 0 nếu cần
    url_len = len(encoded_url)
    if url_len < max_length:
        encoded_url += [0] * (max_length - url_len)
    
    # Chuyển đổi thành mảng numpy và trả về
    return np.array([encoded_url])

### Load Model 

In [55]:
#Best capec: architectureA: 8 - architecture B: 2,3,4,5
#Best csic: architectureA: 6 - architecture B: 
path = r"C:\Users\ASUS\Desktop\Đồ án BMW\NT213.O21.ANTT\CAPEC-dataset\ArchitectA\Testcase8\model.h5"
# path = r"C:\Users\ASUS\Desktop\Đồ án BMW\NT213.O21.ANTT\CSIC-2010-dataset\Architecture B\TestCase[2,3,4,5]\model.h5"
#path = r"C:\Users\ASUS\Desktop\Đồ án BMW\NT213.O21.ANTT\CAPEC-dataset\ArchitectB\Test case [2,3,4,5]\model.h5"
model = load_model(path)

In [56]:
# def predict(model ,url):
#     y_pred = model.predict(url)
#     print(y_pred)
#     return 1 if y_pred < 0.5 else 0
    # return y_pred
    
    
def predict(model ,url):
    y_pred = model.predict(url)
    print(1 - y_pred)
    return 1 if y_pred < 0.5 else 0
    #return y_pred

In [62]:
from http.server import BaseHTTPRequestHandler
from urllib.parse import urljoin
import requests
import re

class SimpleHttpProxy(BaseHTTPRequestHandler):
    proxy_routes = {}
    global_cookies = {}

    @classmethod
    def set_routes(cls, proxy_routes):
        cls.proxy_routes = proxy_routes

    @staticmethod
    def check_url_in_whitelist(url_param, whitelist_file):
        """
        Hàm để kiểm tra xem một URL có trong whitelist hay không.

        Tham số:
        - url_param: URL cần kiểm tra
        - whitelist_file: Tệp whitelist chứa danh sách các URL được phép

        Trả về:
        - True nếu URL tồn tại trong whitelist, False nếu không.
        """
        with open(whitelist_file, "r") as file:
            for line in file:
                whitelist_url = line.strip()
                if url_param == whitelist_url:
                    return True
        return False

    def do_GET(self):
        try:
            proxy_route = self.proxy_routes.get('proxy_route')
            
            if proxy_route:
                target_url = urljoin(proxy_route, self.path)
                
                if self.check_url_in_whitelist(target_url, "whitelist.txt"):
                    y_pred = 0
                elif "dvwa/phpinfo.php" in target_url:
                    y_pred = 0
                else:
                    y_pred = predict(model, load_data(target_url))
                print(f"Prediction cho {target_url}", y_pred)
                if y_pred == 1:
                        self.send_response(403)  # Gửi mã trạng thái 403 Forbidden
                        self.send_header('Content-type', 'text/html')
                        self.end_headers()
                        html_content = '''<!DOCTYPE html>
                                            <html lang="en">
                                            <head>
                                                <meta charset="UTF-8">
                                                <meta name="viewport" content="width=device-width, initial-scale=1.0">
                                                <title>403 Forbidden</title>
                                            </head>
                                            <body>
                                                <h1>403 Forbidden</h1>
                                                <p>Truy cập bị từ chối. Vui lòng liên hệ với quản trị viên.</p>
                                            </body>
                                            </html>'''
                        self.wfile.write(html_content.encode('utf-8'))
                        return  # Dừng xử lý và trả về  
                if self.global_cookies:
                    response = requests.get(target_url, allow_redirects=False, cookies=self.global_cookies)
                else:
                    response = requests.get(target_url, allow_redirects=False)
                
                if response.status_code in [301, 302, 303, 307, 308]:
                    redirect_url = response.headers['Location']
                    response = requests.get(redirect_url)
                
                self.send_response(response.status_code)
                content_type = response.headers.get('Content-Type', 'text/html')
                self.send_header('Content-type', content_type)
                self.end_headers()
                self.wfile.write(response.content)
                
                                
                # self.send_response(response.status_code)
                # content_type = response.headers.get('Content-Type', 'text/html')
                # self.send_header('Content-type', content_type)
                # self.end_headers()
                # self.wfile.write(response.content)
                
            else:
                self.send_error(404, 'Proxy route not found')
        except Exception as e:
            self.send_error(500, str(e))

    def do_POST(self):
        try:
            content_length = int(self.headers['Content-Length'])
            post_data = self.rfile.read(content_length)
            proxy_route = self.proxy_routes.get('proxy_route')
            
            if proxy_route:
                target_url = urljoin(proxy_route, self.path)
                
                if self.check_url_in_whitelist(target_url, "whitelist.txt") and post_data is None:
                    y_pred = 0
                    
                elif "dvwa/phpinfo.php" in target_url:
                    y_pred = 0
                else:
                    #payload_to_pred = target_url.rsplit('/') + '?'+ post_data.decode('utf-8', errors='ignore') 
                    #print("TEST",payload_to_pred)
                    y_pred = predict(model, load_data(target_url+'?'+ post_data.decode('utf-8', errors='ignore')))
                    # Kiểm tra nếu dự đoán vượt quá ngưỡng 0.5
 
                print(f"Prediction cho {target_url+'?'+post_data.decode('utf-8', errors='ignore')}",  y_pred)
                if y_pred == 1:
                        self.send_response(403)  # Gửi mã trạng thái 403 Forbidden
                        self.send_header('Content-type', 'text/html')
                        self.end_headers()
                        html_content = '''<!DOCTYPE html>
                                            <html lang="en">
                                            <head>
                                                <meta charset="UTF-8">
                                                <meta name="viewport" content="width=device-width, initial-scale=1.0">
                                                <title>403 Forbidden</title>
                                            </head>
                                            <body>
                                                <h1>403 Forbidden</h1>
                                                <p>Truy cập bị từ chối. Vui lòng liên hệ với quản trị viên.</p>
                                            </body>
                                            </html>'''
                        self.wfile.write(html_content.encode('utf-8'))
                        return  # Dừng xử lý và trả về              
                
                headers = {"Content-Type": "application/x-www-form-urlencoded"}
                session = requests.Session()
                
                if "/dvwa/login.php" in target_url:
                    response = session.post(target_url, data=post_data, headers=headers, allow_redirects=False)
                    if response.status_code == 302:
                        cookies_dict = requests.utils.dict_from_cookiejar(response.cookies)
                    redirect_url = response.headers.get('Location')
                    if redirect_url:
                        response = session.get("http://192.168.124.173/dvwa/" + redirect_url)
                    self.global_cookies.update(cookies_dict)
                elif "/dvwa/security.php" in target_url:
                    response = session.post(target_url, data=post_data, headers=headers, cookies=self.global_cookies, allow_redirects=False)
                    self.global_cookies['security'] = response.cookies.get_dict().get('security')
                    self.global_cookies.update(self.global_cookies)
                    redirect_url = response.headers.get('Location')
                    if redirect_url:
                        response = session.get("http://192.168.124.173/" + redirect_url, cookies=self.global_cookies)
                else:
                    if self.global_cookies:
                        response = session.post(target_url, data=post_data, headers=headers, cookies=self.global_cookies, allow_redirects=False)
                        if response.status_code == 302:
                            cookies_dict = requests.utils.dict_from_cookiejar(response.cookies)
                        redirect_url = response.headers.get('Location')
                        if redirect_url:
                            response = session.get("http://192.168.124.173/" + redirect_url)
                self.send_response(response.status_code)
                content_type = response.headers.get('Content-Type', 'text/html')
                self.send_header('Content-type', content_type)
                #self.send_header('Content-type', 'text/html')
                self.end_headers()
                self.wfile.write(response.content)
            else:
                self.send_error(404, 'Proxy route not found')
        except Exception as e:
            self.send_error(500, str(e))

def start_proxy_server(port=8080):
    from http.server import HTTPServer
    server_address = ('127.0.0.1', port)
    httpd = HTTPServer(server_address, SimpleHttpProxy)
    print(f'Starting HTTP Proxy server on 127.0.0.1:{port}')
    try:
        httpd.serve_forever()
    except KeyboardInterrupt:
        print('Keyboard interrupt, exiting.')

# Set up proxy routes
SimpleHttpProxy.set_routes({'proxy_route': 'http://192.168.124.173/dvwa/'})
SimpleHttpProxy.set_routes({'proxy_route': 'http://demo.testfire.net/'})

# Start the proxy server
start_proxy_server()

Starting HTTP Proxy server on 127.0.0.1:8080
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
[[0.12057173]]
Prediction cho http://192.168.124.173/dvwa/vulnerabilities/exec/ 0


127.0.0.1 - - [31/May/2024 15:10:26] code 500, message Invalid URL '../../login.php': No scheme supplied. Perhaps you meant https://../../login.php?
127.0.0.1 - - [31/May/2024 15:10:26] "GET http://192.168.124.173/dvwa/vulnerabilities/exec/ HTTP/1.1" 500 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
[[0.208897]]
Prediction cho http://192.168.124.173/dvwa/login.php 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
[[0.11694086]]
Prediction cho http://192.168.124.173/dvwa/dvwa/css/login.css 0


127.0.0.1 - - [31/May/2024 15:10:30] "GET http://192.168.124.173/dvwa/login.php HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2024 15:10:30] "GET http://192.168.124.173/dvwa/dvwa/css/login.css HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
[[0.071015]]
Prediction cho http://192.168.124.173/dvwa/dvwa/images/login_logo.png 0


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
[[0.09400213]]
Prediction cho http://192.168.124.173/dvwa/vulnerabilities/upload/ 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
[[0.08870149]]
Prediction cho http://192.168.124.173/dvwa/dvwa/css/main.css 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step

127.0.0.1 - - [31/May/2024 15:16:42] "GET http://192.168.124.173/dvwa/vulnerabilities/upload/ HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2024 15:16:42] "GET http://192.168.124.173/dvwa/dvwa/css/main.css HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
[[0.06553733]]
Prediction cho http://192.168.124.173/dvwa/dvwa/js/dvwaPage.js 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
[[0.07040137]]
Prediction cho http://192.168.124.173/dvwa/dvwa/images/logo.png 0


127.0.0.1 - - [31/May/2024 15:16:42] "GET http://192.168.124.173/dvwa/dvwa/js/dvwaPage.js HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2024 15:16:42] "GET http://192.168.124.173/dvwa/dvwa/images/logo.png HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2024 15:17:18] code 501, message Unsupported method ('CONNECT')
127.0.0.1 - - [31/May/2024 15:17:18] "CONNECT push.services.mozilla.com:443 HTTP/1.1" 501 -


Keyboard interrupt, exiting.



<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">

<html xmlns="http://www.w3.org/1999/xhtml">

	<head>

		<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />

		<title>Login :: Damn Vulnerable Web Application (DVWA) v1.10 *Development*</title>

		<link rel="stylesheet" type="text/css" href="dvwa/css/login.css" />

	</head>

	<body>

	<div id="wrapper">

	<div id="header">

	<br />

	<p><img src="dvwa/images/login_logo.png" /></p>

	<br />

	</div> <!--<div id="header">-->

	<div id="content">

	<form action="login.php" method="post">

	<fieldset>

			<label for="user">Username</label> <input type="text" class="loginInput" size="20" name="username"><br />


			<label for="pass">Password</label> <input type="password" class="loginInput" AUTOCOMPLETE="off" size="20" name="password"><br />

			<br />

			<p class="submit"><input type="submit" value="Login" name="Login"></p>

	</fieldset>

	<input type='hidden' 

In [ ]:
# # Sử dụng hàm load_data với một URL
# url = "http://demo.testfire.net/search.jsp?query=%3Cscript%3Ealert%281%29%3C%2Fscript%3E"

# loaded_data